In [1]:
%pylab inline
num_samples = 44100
input1 = linspace(-1, 1, num_samples)
input2 = linspace(1, -1, num_samples)

Populating the interactive namespace from numpy and matplotlib


In [6]:
!cat 01_simple_reaction.stride

use DesktopAudio version 1.0

import Generators

signal Value {
    default: 0
}

reaction Adder {
    streams: [
        Value + 10 >> Value;
    ]
}

AudioIn >> Greater() >> Adder();

Value >> AudioOut[0];


In [7]:
out_text = ''

value = 0.0
def add():
    global value
    value += 10
    

for val1, val2 in zip(input1, input2):
    if val1 > val2:
        add()
    out_text += str(value) + "\n" +"0.0\n"

out_file = open("01_simple_reaction.expected", "w")
out_file.write(out_text[:-1])

pass

In [8]:
!cat 02_module_in_reaction.stride

use DesktopAudio version 1.0

import Generators

signal Value {
    default: 0
}

module Add10 {
    ports: [
		mainOutputPort OutputPort {
			block: Output
			domain: OutputPort.domain
		}
		mainInputPort InputPort {
			block: Input
			domain: OutputPort.domain
		}
    ]
    streams: [
        Input + 10 >> Output;
    ]
}

reaction Adder {
    streams: [
        Value >> Add10() >> Value;
    ]
}

AudioIn >> Greater() >> Adder();

Value >> AudioOut[0];


In [9]:
out_text = ''

value = 0.0
def add():
    global value
    value += 10
    

for val1, val2 in zip(input1, input2):
    if val1 > val2:
        add()
    out_text += str(value) + "\n" +"0.0\n"

out_file = open("02_module_in_reaction.expected", "w")
out_file.write(out_text[:-1])

pass

In [10]:
cat 03_constant_switch.stride

use DesktopAudio version 1.0

reaction ConstantSwitch {
	ports: [
		mainOutputPort OutputPort {
			block: ReactionOutput
		}
	]
	streams: [
		1.8 >> ReactionOutput;
	]
}

on >> ConstantSwitch() >> AudioOut[0];
	


In [11]:
out_text = ''

value = 0.0
    

for val1, val2 in zip(input1, input2):
    out_text += str(1.8) + "\n" +"0.0\n"

out_file = open("03_constant_switch.expected", "w")
out_file.write(out_text[:-1])

pass

In [12]:
cat 04_instance_portproperties.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
	]
	blocks: [
		signal Counter { domain: OutputPort.domain }
		reaction HasOuterPortProperty {
			ports: [
				mainOutputPort OutputPort {
					block: ReactionOutput
				}
			]
			streams: [
				Counter + 1 >> Counter;
				Counter / OutputPort.rate >> ReactionOutput;
			]
		}
			
	]
	streams: [
		on >> HasOuterPortProperty() >> Output;
	]
	
}
	
Test() >> AudioOut[0];


In [13]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    
    out_text += str(counter/44100.0) + "\n" +"0.0\n"

out_file = open("04_instance_portproperties.expected", "w")
out_file.write(out_text[:-1])

pass

In [14]:
cat 05_outer_instance_portproperties.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainOutputPort OutputPort {
			block: Output
		}
	]
	blocks: [
		signal Counter {}
		reaction HasOuterPortProperty {
			ports: [
				mainOutputPort OutputPort {
					block: ReactionOutput
				}
			]
			streams: [
				Counter + 1 >> Counter;
				Counter / OutputPort.rate >> ReactionOutput;
			]
		}
			
	]
	streams: [
		on >> HasOuterPortProperty() >> Output;
	]
	
}
	
Test() >> AudioOut[0];


In [15]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    out_text += str(counter/44100.0) + "\n" +"0.0\n"

out_file = open("05_outer_instance_portproperties.expected", "w")
out_file.write(out_text[:-1])

pass

In [16]:
cat 06_used_only_in_reaction.stride

use DesktopAudio version 1.0

module InReaction {
    ports: [
        mainOutputPort OutputPort {
            block: Output
        }
        mainInputPort InputPort {
            block: Input
        }
    ]
	blocks: [
        reaction PassThruReaction {
			ports: [
				mainOutputPort OutputPort {
					block: Output
				}
			]
            streams:    [
                Input >> Output;
            ]
        }
    ]
    streams: [
        on >> PassThruReaction() >> Output;
    ]
}


AudioIn[0] >> InReaction () >> AudioOut;

In [17]:
out_text = ''

counter = 0.0
    

for val1, val2 in zip(input1, input2):
    counter += 1.0
    out_text += str(val1) + "\n" + str(val1) + "\n"

out_file = open("06_used_only_in_reaction.expected", "w")
out_file.write(out_text[:-1])

pass

In [18]:
cat 07_reaction_in_module.stride

cat: 07_property_port_in_reaction.stride: No such file or directory


In [21]:
out_text = ''

value = 0

for val1, val2 in zip(input1, input2):
    if val1 > 0:
        value += 1
    out_text += str(value) + "\n0.0\n"

out_file = open("07_reaction_in_module.expected", "w")
out_file.write(out_text[:-1])

pass

In [20]:
cat 08_reaction_recursive_reference.stride

use DesktopAudio version 1.0

module Test {
	ports: [
		mainInputPort InputPort {
			block: Input	
		}	
		mainOutputPort OutputPort {
			block: Output	
		}
	]
	blocks: [
		signal Dummy {}
		signal Dummy2 { domain: OutputPort.domain}
		reaction Process {
			streams: [
				Dummy >> Dummy2;
				on >> ProcessInner();
			]
		}
		reaction ProcessInner {
			streams: [
				Input >> Output;
			]
		}
	]
	streams: [
		on >> Process();
	]
}

AudioIn[0] >> Test() >> AudioOut[0];